# Importamos bibliotecas
Descargamos la biblioteca de transformers de keras, sacreblue para la evaluación y tqdm para ver el avance de la evaluación

In [ ]:
pip install keras-transformer

In [66]:
pip install sacrebleu

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 90 kB 927 kB/s eta 0:00:01
     |████████████████████████████████| 764 kB 316 kB/s eta 0:00:01
ERROR: Error while checking for conflicts. Please file an issue on pip's issue tracker: https://github.com/pypa/pip/issues/new
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3012, in _parsed_pkg_info
    return self._pkg_info
  File "/usr/local/lib/python3.8/dist-packages/pi

In [71]:
pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 76 kB 737 kB/s eta 0:00:01
ERROR: Error while checking for conflicts. Please file an issue on pip's issue tracker: https://github.com/pypa/pip/issues/new
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3012, in _parsed_pkg_info
    return self._pkg_info
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    

# Datos
Preprocesamiento de datos

In [1]:
# Biblioteca para usar transformers
from keras_transformer import get_model, decode
from pickle import load

# Ponemos la semilla en 0 para replicar
import numpy as np
np.random.seed(0)

In [2]:
# Cargamos el archivo con los tokens paralelos
filenameParalelo = 'corpus/tokens_paralelos.txt'
with open(filenameParalelo) as file:
    datos = file.readlines()

In [3]:
def separarCorpus(corpus):
    """
    Nos permite leer el archivo de tokens y separar de manera
    ordenada para tener el corpus en español y nahuatl
    
    Parámetros
    corpus: Lista de oraciones paralelas
    
    return lista de oraciones en nahuatl y en español
    """
    nah = list()
    esp = list()
    for oracion in corpus:
        if oracion != "\t\n":
            sep = oracion[:-1].split("\t")
            nah.append(sep[0].split(" "))
            esp.append(sep[1].split(" "))
    return nah, esp

In [4]:
# Variables globales
nah, esp = separarCorpus(datos)

In [5]:
# Variables globales
LIMITE = 512
nah = list(map(lambda x: x[:LIMITE], nah))
esp = list(map(lambda x: x[:LIMITE], esp))

In [6]:
len(nah[0])

65

In [7]:
from collections import defaultdict

def create_dict():
    """
    Crea un diccionario cuya llave por default de 
    elementos nuevos, es el tamaño del diccionario
    
    return defaultdict
    """
    token_dict = defaultdict()
    token_dict.default_factory = lambda: len(token_dict)
    return token_dict

# Variables globales
PAD = 0
START = 1
END = 2

def build_token_dict(token_list):
    """
    Crea el diccionario para un vocabulario
    
    Parámetros
    token_list: Lista de oraciones tokenizadas
    
    return diccionario de tokens y su mapeo a numeros
    """
    token_dict = create_dict()
    token_dict['<PAD>'] = PAD
    token_dict['<START>'] = START
    token_dict['<END>'] = END

    for tokens in token_list:
        for token in tokens:
          if token not in token_dict:
            token_dict[token]
    return token_dict

In [8]:
# Variables globales
source_tokens = nah
target_tokens = esp

In [9]:
# Variables globales
source_token_dict = build_token_dict(source_tokens)                     #Nahualt
target_token_dict = build_token_dict(target_tokens)                     #Español
target_token_dict_inv = {v:k for k,v in target_token_dict.items()}      #Inverso para la salida

In [51]:
# Diccionario del source para decodificar
source_token_dict_inv = {v:k for k,v in source_token_dict.items()}      #Inverso para la salida

In [12]:
len(source_token_dict)

418

In [13]:
len(target_token_dict)

439

In [31]:
# Agregar start y end a cada frase del set de entrenamiento
encoder_tokens = [['<START>'] + tokens + ['<END>'] for tokens in source_tokens]
decoder_tokens = [['<START>'] + tokens + ['<END>'] for tokens in target_tokens]
output_tokens = [tokens + ['<END>'] for tokens in target_tokens]

source_max_len = max(map(len, encoder_tokens))      #Maximo de palabras en frase de ingles
target_max_len = max(map(len, decoder_tokens))      #Maximo de palabras en frase de español

#Llenar con padding las frases para que sean del mismo tamaño
encoder_tokens = [tokens + ['<PAD>']*(source_max_len-len(tokens)) for tokens in encoder_tokens]
decoder_tokens = [tokens + ['<PAD>']*(target_max_len-len(tokens)) for tokens in decoder_tokens]
output_tokens = [tokens + ['<PAD>']*(target_max_len-len(tokens)) for tokens in output_tokens]

In [32]:
print(encoder_tokens[2])

['<START>', 'no@@', 'n', 'tla@@', 'ka@@', 'h', 'ma@@', 'ya@@', 'ki@@', 'ah', 'amo', 'ki@@', 'x@@', 'to@@', 'no@@', 'n', 'no@@', 'n', 'in', 'tz@@', 'o@@', 'tz@@', 'o@@', 'ma', 'cui@@', 'ka', 'in', 'pan@@', 'ta@@', 'l@@', 'on', 'chi@@', 'pa@@', 'hua@@', 'k', 'on@@', 'tz@@', 'o', 'in@@', 'ka@@', 'z@@', 'on', 'de', 'man@@', 'ta', 'ko@@', 'to@@', 'n@@', 'tle@@', 'h', 'chi@@', 'pa@@', 'hua@@', 'k', 'o', 'zan@@', 'tle@@', 'li@@', 'hui@@', 'z', 'in', 'xo@@', 'm@@', 'p@@', 'le@@', 'lo@@', 'h', 'tla', 'cue@@', 'cue@@', 'zo@@', 'tl', 'ka', 'zo@@', 'ya@@', 'tl', 'huan', 'in', 'te@@', 'ka@@', 'k', 'de', 'cue@@', 'ro', 'ki@@', 'on', 'o@@', 'ma@@', 'ya@@', 'ki@@', 'aya@@', 'h', 'no@@', 'n', 'i@@', 'hue@@', 'ka', 'tla@@', 'ka@@', 'a@@', 'x@@', 'k@@', 'an', 'no@@', 'n', 'in', 'tz@@', 'o@@', 'tz@@', 'o@@', 'ma@@', 'huan', 'yo@@', 'po@@', 'li@@', 'ke@@', 'h@@', 'a@@', 'x@@', 'k@@', 'an', 'mo', 'cue@@', 'h', 'za', 'no@@', 'n', 'xo@@', 'm@@', 'p@@', 'le@@', 'lo@@', 'h', 'ma@@', 'z@@', 'ke', 'ma@@', 'ki@@',

In [33]:
# Se mapean a numeros los tokens de cada oración
encoder_input = [list(map(lambda x: source_token_dict[x], tokens)) for tokens in encoder_tokens]
decoder_input = [list(map(lambda x: target_token_dict[x], tokens)) for tokens in decoder_tokens]
output_decoded = [list(map(lambda x: [target_token_dict[x]], tokens)) for tokens in output_tokens]

print(encoder_input[100])

[1, 18, 80, 58, 106, 60, 130, 18, 100, 78, 33, 15, 43, 130, 22, 117, 78, 33, 15, 82, 205, 169, 125, 82, 106, 106, 60, 154, 227, 52, 183, 238, 140, 82, 125, 82, 161, 15, 133, 128, 18, 20, 36, 248, 29, 7, 53, 9, 106, 148, 37, 140, 18, 71, 146, 36, 16, 20, 65, 106, 148, 65, 136, 32, 82, 231, 7, 128, 7, 98, 148, 37, 140, 18, 56, 42, 15, 82, 205, 36, 16, 20, 65, 106, 148, 164, 36, 16, 119, 130, 58, 51, 107, 65, 243, 98, 128, 194, 37, 140, 18, 56, 42, 231, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [39]:
# Variable global
test = int(len(nah)*0.9)

# Dividimos en conjunto de test y train
encoder_input_test = encoder_input[test:]
decoder_input_test = decoder_input[test:]
output_decoded_test = output_decoded[test:]

encoder_input = encoder_input[:test]
decoder_input = decoder_input[:test]
output_decoded = output_decoded[:test]

In [54]:
len(encoder_input_test)

3752

In [55]:
len(encoder_input)

33764

# Configuración del entorno
Comandos para poder trabajar con ciertas gpus

In [34]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  8


In [35]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.set_visible_devices(gpus[5], 'GPU')
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

8 Physical GPUs, 1 Logical GPU


In [36]:
# %env CUDA_DEVICE_ORDER="PCI_BUS_ID"
# %env CUDA_VISIBLE_DEVICES="5"
# ! nvidia-smi

In [37]:
# Variable global
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:4', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:5', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:6', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:7', device_type='GPU')]

# Creación del modelo

In [38]:
# Crear la red transformer
model = get_model(
    token_num = max(len(source_token_dict),len(target_token_dict)),
    embed_dim = 32,
    encoder_num = 2,
    decoder_num = 2,
    head_num = 4,
    hidden_dim = 128,
    dropout_rate = 0.05,
    use_same_embed = False,
)
model.compile('adam', 'sparse_categorical_crossentropy')
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 Encoder-Input (InputLayer)     [(None, None)]       0           []                               
                                                                                                  
 Encoder-Token-Embedding (Embed  [(None, None, 32),  14048       ['Encoder-Input[0][0]']          
 dingRet)                        (439, 32)]                                                       
                                                                                                  
 Encoder-Embedding (TrigPosEmbe  (None, None, 32)    0           ['Encoder-Token-Embedding[0][0]']
 dding)                                                                                           
                                                                                            

# Entrenamiento

In [49]:
# Entrenamiento

# Se crea la carpeta para una versión del modelo
import os
ruta_modelo = "model/v9-1/"
if not os.path.exists(ruta_modelo):
  os.mkdir(ruta_modelo)

# Cargar modelo
#from keras.models import load_model

#fileWeights = 'model/v5-1/pesos.h5'
#model.load_weights(fileWeights)

x = [np.array(encoder_input), np.array(decoder_input)]
y = np.array(output_decoded)

# Guardamos la arquitectura
model.save(ruta_modelo + "modelo.h5")

# Variable para iteraciones
acc = 257
while (acc < 500):
  print('Epoca:',acc + 1)
  model.fit(x,y, epochs = 1, batch_size=32)
  model.save_weights(ruta_modelo + "pesos.h5")
  acc = acc + 1

Epoca: 258
1056/1056 [==============================] - 88s 77ms/step - loss: 0.1786
Epoca: 259
1056/1056 [==============================] - 81s 77ms/step - loss: 0.1786
Epoca: 260
1056/1056 [==============================] - 81s 77ms/step - loss: 0.1786
Epoca: 261
1056/1056 [==============================] - 81s 77ms/step - loss: 0.1786
Epoca: 262
1056/1056 [==============================] - 81s 77ms/step - loss: 0.1785
Epoca: 263
1056/1056 [==============================] - 82s 77ms/step - loss: 0.1786
Epoca: 264
1056/1056 [==============================] - 82s 77ms/step - loss: 0.1785
Epoca: 265
1056/1056 [==============================] - 81s 77ms/step - loss: 0.1785
Epoca: 266
1056/1056 [==============================] - 81s 77ms/step - loss: 0.1784
Epoca: 267
1056/1056 [==============================] - 81s 77ms/step - loss: 0.1785
Epoca: 268
1056/1056 [==============================] - 81s 77ms/step - loss: 0.1784
Epoca: 269
1056/1056 [==============================] - 81s 77ms/

In [50]:
# Guardar 
import os
ruta_modelo = "model/v9-1/"
if not os.path.exists(ruta_modelo):
  os.mkdir(ruta_modelo)

model.save(ruta_modelo+"modelo.h5")
model.save_weights(ruta_modelo+"pesos.h5")

In [40]:
# Cargar modelo
ruta_modelo = "model/v9-1/"
modelo = "modelo.h5"
pesos = 'pesos.h5'
model.load_weights(ruta_modelo+pesos)

# Predicciones

In [69]:
def decode_source(oracion):
    """
    Toma una oración del lenguaje source y la decodifica
    
    Parámetros
    oracion: Oración del lenguaje source en numeros/tokens
    
    return oración decodificada
    """
    oracion = ' '.join(map(lambda x: source_token_dict_inv[x], oracion)).replace(" <PAD>","")
    oracion = oracion.replace("@@ ","").replace(" <END>","").replace("<START>","")
    return oracion

def decode_translated(oracion):
    """
    Toma una oración del lenguaje target y la decodifica
    
    Parámetros
    oracion: Oración del lenguaje target en numeros/tokens
    
    return oración decodificada
    """
    oracion = ' '.join(map(lambda x: target_token_dict_inv[x], oracion)).replace(" <PAD>","")
    oracion = oracion.replace("@@ ","").replace(" <END>","").replace("<START>","")
    return oracion
    
def translate(indice):
    """
    Traducción de una oración
    
    Parámetros
    indice: indice en la lista de test 
    """
    sentence = encoder_input_test[indice]
    original = decoder_input_test[indice]
    decoded = decode(
      model, 
      sentence, 
      start_token = target_token_dict['<START>'],
      end_token = target_token_dict['<END>'],
      pad_token = target_token_dict['<PAD>']
    )

    print('Frase original:\t\t {}'.format(decode_source(sentence)))
    print('Traducción:\t\t {}'.format(decode_translated(decoded)))
    print('Traducción original:\t {}'.format(decode_translated(original)))
    
def translate_bleu(indice):
    """
    Traducción de una oración
    
    Parámetros
    indice: indice en la lista de test 
    
    return oración-traducción predecida y original
    """
    sentence = encoder_input_test[indice]
    original = decoder_input_test[indice]
    decoded = decode(
      model, 
      sentence, 
      start_token = target_token_dict['<START>'],
      end_token = target_token_dict['<END>'],
      pad_token = target_token_dict['<PAD>']
    )

    #print('Traducción:\t\t {}'.format())
    #print('Traducción original:\t {}'.format())
    
    return decode_translated(decoded), decode_translated(original)
    

In [64]:
translate(120)

Frase original:		  zeki kitektinemij kampa onkak uan kinamakaj zakapoaztan uan no cuezalan
Traducción:		  unos pájaros también hay en donde hay en los palos y se puede se puede haber
Traducción original:	  algunos lo andan cortando donde lo hay y lo venden en zacapoaxtla y también en cuetzalan


In [44]:
# Vemos cual es el valor con más tokens
maximo = 0
long = 0
for i in range(len(encoder_input_test)):
    val = len(encoder_input_test[i])-encoder_input_test[i].count(0)
    if val > long:
        maximo = i
        long = val

In [67]:
from sacrebleu.metrics import BLEU

In [72]:
from tqdm import tqdm

# Variables globales
traducciones = list()
originales = list()

for i in tqdm(range(len(encoder_input_test))):
    tra, orig = translate_bleu(i)
    traducciones.append(tra)
    originales.append(orig)

100%|██████████| 3752/3752 [2:37:41<00:00,  2.52s/it]  


In [73]:
# Variable global
bleu = BLEU()
bleu.corpus_score(traducciones, [originales])

BLEU = 2.59 18.2/4.0/1.2/0.5 (BP = 1.000 ratio = 1.065 hyp_len = 88840 ref_len = 83431)